In [1]:
import torch
import nip

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

DEVICE = 'cuda:0'

In [2]:
cpnt_path = '/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/T256_L10_V2_K3_lbits10_LowBitSym_qtip.pth'
qmodel = torch.load(cpnt_path)
tokenizer=qlib.load_tokenizer('Llama2-7b-hf')
qmodel = qmodel.to(DEVICE)

/tmp/ipykernel_298099/1711781055.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  qmodel = torch.load(cpnt_path)


In [3]:
# trellis_layer = qmodel.get_submodule('model.layers.7.self_attn.q_proj')
# test_input = torch.randn(4, 4096, 4096).half().to('cuda:0')

# %%timeit
# with torch.amp.autocast('cuda', dtype=torch.float16):
#     with torch.no_grad():
#         trellis_layer(test_input)

# full 48.8 ms
# wo incoh 44.8 ms
# wo linear 26.8 ms
# wo linear and incoh 22.8 ms
# full 1mad 61.6 ms
# full 1mad_short 73.3 ms
# full 1mad_mid 62.1 ms

In [4]:
dataloader = qlib.QATDataset(
    config=nip.load('/home/msst/repo/Quantization/configs/data/wikitext_test_seqlen4096.yaml'),
    tokenizer=tokenizer
).get_dataloader()


with torch.amp.autocast('cuda', dtype=torch.float16):
    res = qlib.evaluate(qmodel, dataloader, print_times=25)
    print(res)

  7%|▋         | 2/28 [00:30<06:38, 15.32s/it]

5.978120393454391


 11%|█         | 3/28 [00:45<06:22, 15.29s/it]

6.34252482173532


 14%|█▍        | 4/28 [01:01<06:05, 15.24s/it]

...

6.2616833380811


 96%|█████████▋| 27/28 [06:49<00:15, 15.17s/it]

6.2491666876655


100%|██████████| 28/28 [07:02<00:00, 15.09s/it]

6.2647661921771665
6.2647661921771665
